In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-10 12:28:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.01MB/s    in 0.3s    

2025-06-10 12:28:03 (4.01 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt','r',encoding='utf-8') as f:
  text=f.read()

In [3]:
print("Length of Tiny shakesphear is ",len(text))

Length of Tiny shakesphear is  1115394


In [4]:
print(text[:986])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst 


In [5]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print("No of char ",vocab_size)



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
No of char  65


In [6]:
stoi={}
itos={}

for i,ch in enumerate(chars):
  stoi[ch]=i
  itos[i]=ch

encode=lambda s:[stoi[ch] for ch in s ]
decode=lambda l:''.join(itos[i] for i in l)

print(encode("Udayyy says Hi"))
print(decode(encode("Udayyy says Hi")))


[33, 42, 39, 63, 63, 63, 1, 57, 39, 63, 57, 1, 20, 47]
Udayyy says Hi


In [7]:
import torch

data=torch.tensor(encode(text),dtype=torch.long)

In [8]:
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
n=int(0.9*len(data))

train=data[:n]
val=data[n:]

In [10]:
blockSize=8

train[:blockSize+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x=train[:blockSize]
y=train[1:blockSize+1]

for t in range(blockSize):
  context=x[:t+1]
  target=y[t]

  print(f"for context {context} target is {target}")

for context tensor([18]) target is 47
for context tensor([18, 47]) target is 56
for context tensor([18, 47, 56]) target is 57
for context tensor([18, 47, 56, 57]) target is 58
for context tensor([18, 47, 56, 57, 58]) target is 1
for context tensor([18, 47, 56, 57, 58,  1]) target is 15
for context tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
for context tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [12]:
torch.manual_seed(1337)
blockSize=8
batchSize=4

def getBatch(split):
  data=train if split=='train' else val

  idx=torch.randint(len(data)-blockSize,(batchSize,))
  x=torch.stack([data[i:i+blockSize] for i in idx])
  y=torch.stack([data[i+1:i+blockSize+1] for i in idx])

  return x,y

xb,yb=getBatch('train')

print("input:")
print(xb.shape)
print(xb)
print()
print("target:")
print(yb.shape)
print(yb)



input:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

target:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguagemodel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx,targets=None):

    logits=self.token_embedding_table(idx)

    if targets==None:
      loss=None
    else:
      B,T,C=logits.shape
      logits=logits.reshape(B*T,C)
      targets=targets.reshape(B*T)

      loss=F.cross_entropy(logits,targets)
    return logits,loss

  def generate(self,idx,max_token):

    for _ in range(max_token):
      logits,loss=self.forward(idx)

      logits=logits[:,-1,:]

      prob=F.softmax(logits,dim = -1)

      idx_next=torch.multinomial(prob,num_samples=1)

      idx=torch.cat((idx,idx_next),dim=1)

    return idx

m=BigramLanguagemodel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx=torch.zeros(1,1,dtype=torch.long),max_token=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [14]:

optimzer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [15]:
batchSize=32

for _ in range(30000):
  xb,yb=getBatch('train')

  logits,loss=m(xb,yb)
  optimzer.zero_grad(set_to_none=True)
  loss.backward()
  optimzer.step()

print(loss)

tensor(2.3530, grad_fn=<NllLossBackward0>)


In [16]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_token=100)[0].tolist()))


Sthitambst lay d, ango thinouggof d RLUTous ds.
O:
NNoucoferesainteiss,
ENCh;
Heme LEEN:
RYor sprear


In [27]:
a=torch.tril(torch.ones(3,3))
a=a/(a.sum(1,keepdim=True))
b=torch.randint(0,10,(3,2)).float()
c=a@b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[0., 4.],
        [0., 3.],
        [8., 4.]])
tensor([[0.0000, 4.0000],
        [0.0000, 3.5000],
        [2.6667, 3.6667]])


In [30]:
torch.manual_seed(1337)

B,T,C=4,8,2

x=torch.randn(B,T,C)
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])

In [33]:
xbow=torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1]
    xbow[b,t]=torch.mean(xprev,0)
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])